In [ ]:
import cv2
import numpy as np
import os

input_folder = 'input path'
output_folder = 'output path'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

input_files = [file for file in os.listdir(input_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

for file in input_files:
    image = cv2.imread(os.path.join(input_folder, file))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lower_black = np.array([0])
    upper_black = np.array([30])
    black_mask = cv2.inRange(gray, lower_black, upper_black)
    contours, _ = cv2.findContours(black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_line = None
    largest_line_length = 0
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(largest_contour, closed=True)
        for contour in contours:
            for i in range(len(contour)):
                for j in range(i, len(contour)):
                    point1 = contour[i][0]
                    point2 = contour[j][0]
                    line_length = np.linalg.norm(np.array(point1) - np.array(point2))
                    if line_length > largest_line_length and line_length <= perimeter:
                        largest_line_length = line_length
                        largest_line = (point1, point2)
        image_with_line = image.copy()
        cv2.line(image_with_line, largest_line[0], largest_line[1], (0, 0, 255), 2)
        text = f"Major Axis Length = {largest_line_length:.2f} pixels"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_color = (0, 0, 255)
        font_thickness = 2
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = 10
        text_y = 20
        cv2.putText(image_with_line, text, (text_x, text_y), font, font_scale, font_color, font_thickness)
        output_path = os.path.join(output_folder, file)
        cv2.imwrite(output_path, image_with_line)
